In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from scipy.optimize import curve_fit

In [ ]:
params = {'legend.fontsize': 17,
          'figure.figsize': (15, 9.3),
          'axes.labelsize': 24,
          'axes.titlesize': 24,
          'xtick.labelsize': 22,
          'ytick.labelsize': 22}
plt.rcParams.update(params)
params = ['x', 'y', 't', 'E', 'azi']

In [ ]:
class spike_gen(stats.rv_continuous):
    def pdf(self, x, theta):
        return np.exp(-2*np.abs(x-theta))
    def rvs(self, theta):
        out = []
        while len(out) < len(theta):
            P = np.random.rand(len(theta)-len(out))*20 - 10
            p = self.pdf(P, 0)
            w = np.random.rand(len(theta)-len(out))
            out.extend(P[w<p])
        return np.array(out) + theta
spike = spike_gen()

In [ ]:
def LLH(theta, x):
    #return -np.log(stats.poisson.pmf(x, theta))
    return -np.log(stats.norm.pdf(x, theta, 2)) #+stats.norm.pdf(x+1, theta, 2)
    #return -np.log(spike.pdf(x, theta))

In [ ]:
T = np.linspace(1,100,100)
plt.plot(T, LLH(T, 50), label=r'$x=50$')
plt.plot(T, LLH(T, 5), label=r'$x=5$')
plt.legend()
plt.title('Poisson distribution')
plt.xlabel(r'$\theta$')
plt.ylabel('-LLH')
plt.ylim(-1, 100)
#plt.savefig('../../plots/uncertainties/from_dLLH/1D_toy/llh_scans_poisson', bbox_inches='tight')

In [ ]:
theta = np.random.uniform(5, 9, 1000000)
#x = stats.poisson.rvs(theta) # = theta_bf
#x = stats.norm.rvs(theta, 2)
x = spike.rvs(theta)

resi = x-theta
resi_abs = np.abs(resi)

#x2 = stats.norm.rvs(theta, 2)
#resi2 = (x+x2)/2-theta

In [ ]:
plt.hist(resi, 200, label='std = %.2f'%(np.std(resi)), density=True)
#plt.hist(resi2, np.linspace(min(resi), max(resi), 200), label='std = %.2f'%(np.std(resi2)), alpha=0.5)
plt.plot(np.linspace(-2,2,200), spike.pdf(np.linspace(-2,2,200), 0))
plt.legend()
plt.title(r'Normal distribution ($\theta_{true}$ from 5-9)')
plt.xlabel(r'$\theta_{reco}-\theta_{true}$')
plt.ylabel('#events')
#plt.savefig('../../plots/uncertainties/from_dLLH/1D_toy/resi_normal_low', bbox_inches='tight')

In [ ]:
d_llh = LLH(theta, x)-LLH(x, x) #x=theta_bf

d_llh_68 = np.percentile(d_llh, 68)
d_llh_68

In [ ]:
plt.hist(d_llh, 200)
plt.axvline(d_llh_68, label='68 percentile = %.2f'%(d_llh_68), color='black')
plt.legend()
plt.title(r'Normal distribution ($\theta_{true}$ from 5-9)') # recoed with normal
plt.xlabel(r'$\Delta LLH$')
plt.ylabel('#events')
plt.yscale('log')
#plt.savefig('../../plots/uncertainties/from_dLLH/1D_toy/dllh_normal_low', bbox_inches='tight')

In [ ]:
dLLH_range = np.logspace(-2, 0.2, 30)
dLLH_center = (dLLH_range[1:]+dLLH_range[:-1])/2
medi = []
for i in range(len(dLLH_range)-1):
    test = np.abs(resi)[(d_llh > dLLH_range[i]) & (d_llh < dLLH_range[i+1])]
    if len(test) == 0: test = np.zeros(1)
    medi.append(np.mean(test))

In [ ]:
def func(x, a):
    return a * np.sqrt(x)

popt, pcov = curve_fit(func, dLLH_center, medi)

unc_68 = func(d_llh_68, popt[0])
unc_68_2 = func(2*d_llh_68, popt[0])
unc_68, unc_68_2

In [ ]:
plt.scatter(d_llh, resi_abs)
plt.scatter(dLLH_center, medi, color='r', label='means')
plt.plot(dLLH_center, func(dLLH_center, popt[0]), color='r', label=r'$a\cdot\sqrt{x}$ fit to mean')
plt.axvline(d_llh_68, color='red', linestyle='--', label='68 percentile (%.2f, %.2f)'%(d_llh_68, unc_68))
plt.axhline(unc_68, color='red', linestyle='--')
plt.legend(loc='upper left')
plt.title(r'Normal distribution ($\theta_{true}$ from 5-9)') # recoed with normal
plt.xlabel(r'$\Delta LLH$')
plt.ylabel(r'$|\theta_{reco}-\theta_{true}|$')
#plt.savefig('../../plots/uncertainties/from_dLLH/1D_toy/unc_est_normal_low', bbox_inches='tight')